## Import

In [1]:
import numpy as np
import pandas as pd

## Exploring data

### Read raw data from file

In [2]:
raw_df = pd.read_csv("../data/raw/data.csv")

In [3]:
raw_df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3548,s3549,Movie,Feo pero sabroso,Fernando Ayllón,"Iván Marín, Lina Cardona, Carlos Barbosa, Ana ...",Colombia,"August 30, 2019",2018,TV-MA,91 min,"Comedies, International Movies",When an unattractive man gets engaged to a bea...
8508,s8509,Movie,The Similars,Isaac Ezban,"Gustavo Sánchez Parra, Cassandra Ciangherotti,...",Mexico,"January 15, 2017",2015,NR,90 min,"Horror Movies, Independent Movies, Internation...",Stranded in a transit station one stormy night...
4087,s4088,Movie,Paris Is Us,Elisabeth Vogler,"Noémie Schmidt, Grégoire Isvarine, Marie Motte...",France,"February 22, 2019",2019,TV-MA,84 min,"Dramas, Independent Movies, International Movies",Amid a turbulent romance and rising tensions i...
6724,s6725,TV Show,Extras,NaN,"Ricky Gervais, Stephen Merchant, Ashley Jensen...","United Kingdom, United States","March 31, 2017",2006,TV-MA,2 Seasons,"British TV Shows, TV Comedies","Grasping hungrily for stardom, bit actor Andy ..."
3585,s3586,TV Show,The King's Avatar,NaN,"Yang Yang, Jiang Shuying, Lai Yumeng, Li Muche...",China,"August 16, 2019",2019,TV-MA,1 Season,"International TV Shows, TV Action & Adventure,...",When an elite gamer is forced out of his profe...


We could drop the show_id and description column because they are not useful for our analysis.

In [4]:
raw_df.drop(columns = ['show_id', 'description'], inplace = True)

### How many rows and how many columns does the raw data have?

In [5]:
shape = raw_df.shape
print(f"Data has {shape[0]} rows and {shape[1]} columns")

Data has 8807 rows and 10 columns


### What is the meaning of each row?

Each row represents information about a movie/ tv show on Netflix from 2008 to mid 2021.

### Are there duplicated rows?

In [6]:
unique_rows = raw_df.drop_duplicates().shape[0] - shape[0]
print(f'Number of duplicate rows: {unique_rows}')

Number of duplicate rows: 0


### What is the meaning of each column?

| Column Name | Description |
| --- | --- |
| show_id | Unique ID for each movie/ tv show |
| type | Type of the movie/ tv show (Movie or TV Show) |
| title | Title of the movie/ tv show |
| director | Director of the movie/ tv show |
| cast | Cast of the movie/ tv show |
| country | Country where the movie/ tv show was produced |
| date_added | Date when the movie/ tv show was added to Netflix |
| release_year | Year when the movie/ tv show was released |
| rating | Rating of the movie/ tv show |
| duration | Duration of the movie/ tv show |
| listed_in | Genre of the movie/ tv show |
| description | Description of the movie/ tv show |

### What is the current data type of each column? Are there columns having inappropriate data types?


In [7]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8807 non-null   object
 1   title         8807 non-null   object
 2   director      6173 non-null   object
 3   cast          7982 non-null   object
 4   country       7976 non-null   object
 5   date_added    8797 non-null   object
 6   release_year  8807 non-null   int64 
 7   rating        8803 non-null   object
 8   duration      8804 non-null   object
 9   listed_in     8807 non-null   object
dtypes: int64(1), object(9)
memory usage: 688.2+ KB


We have null values in director, cast, country, date_added, rating and duration columns.

The reason why we don't want to drop the null values in country column is because it could provide some interesting points.

In [8]:
raw_df['director'] = raw_df['director'].fillna('Unknown')
raw_df['cast'] = raw_df['cast'].fillna('Unknown')
raw_df['country'] = raw_df['country'].fillna('United States') #Because Netflix is based in USA

raw_df.dropna(inplace=True)

In [9]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8790 entries, 0 to 8806
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8790 non-null   object
 1   title         8790 non-null   object
 2   director      8790 non-null   object
 3   cast          8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 755.4+ KB


### Convert data type of date_added column to datetime

In [10]:
raw_df['date_added'] = pd.to_datetime(raw_df['date_added'].str.strip())
raw_df['year_added'] = raw_df['date_added'].dt.year
raw_df['month_added'] = raw_df['date_added'].dt.month

### With each numerical column, how are values distributed?

- Percentage of missing values?

In [11]:
raw_df.select_dtypes('number').isna().sum() / len(raw_df)

release_year    0.0
year_added      0.0
month_added     0.0
dtype: float64

- Describe the values.

In [12]:
raw_df.select_dtypes('number').describe()

,release_year,year_added,month_added
count,8790.000000,8790.000000,8790.000000
mean,2014.183163,2018.873606,6.655859
std,8.825466,1.573568,3.436103
min,1925.000000,2008.000000,1.000000
25%,2013.000000,2018.000000,4.000000
50%,2017.000000,2019.000000,7.000000
75%,2019.000000,2020.000000,10.000000
max,2021.000000,2021.000000,12.000000


All values seem to be normal.

### With each categorical column, how are values distributed?

### Missing ratios of categorical columns

In [13]:
df_copy = raw_df.copy()
df_copy = df_copy.drop(axis = 1, columns=['date_added', 'release_year', 'year_added', 'month_added'])
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def num_values(s):
    s = s.str.split(';')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.str.split(';')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

cat_col_info_df = df_copy.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,type,title,director,cast,country,rating,duration,listed_in
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
num_values,2,8790,4527,7679,748,14,220,513
value_ratios,"{'Movie': 69.7, 'TV Show': 30.3}","{'Dick Johnson Is Dead': 0.0, 'Cooked': 0.0, '...","{'Unknown': 29.8, 'Rajiv Chilaka': 0.2, 'Raúl ...","{'Unknown': 9.4, 'David Attenborough': 0.2, 'V...","{'United States': 41.4, 'India': 11.1, 'United...","{'TV-MA': 36.5, 'TV-14': 24.5, 'TV-PG': 9.8, '...","{'1 Season': 20.4, '2 Seasons': 4.8, '3 Season...","{'Dramas, International Movies': 4.1, 'Documen..."


### Save processed data

In [14]:
raw_df.to_csv('../data/processed/data.csv')